# VIX-Adjusted Momentum (US)

Replicating [VIX-Adjusted Momentum](https://cssanalytics.wordpress.com/2014/07/29/vix-adjusted-momentum/)

In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

indices <- Indices()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.1
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘PerformanceAnalytics’

The following object is masked from ‘package:graphics’:

    legend

Registering fonts with R

Attaching package: ‘gridExtra’

The following object is masked from ‘package:dplyr’:

    combine


Attaching package

In [2]:
startDate <- as.Date('1990-01-02')
endDate <- as.Date('2019-06-30')

vixDt <- indices$YahooFinanceTimeSeries() %>%
    filter(NAME == '^VIX' & TIME_STAMP >= startDate & TIME_STAMP <= endDate) %>%
    select(TIME_STAMP, CLOSE) %>%
    collect()
    
sp500Dt <- indices$YahooFinanceTimeSeries() %>%
    filter(NAME == '^GSPC' & TIME_STAMP >= startDate & TIME_STAMP <= endDate) %>%
    select(TIME_STAMP, CLOSE) %>%
    collect()

allXts <- merge(sp500Xts <- xts(sp500Dt$CLOSE, sp500Dt$TIME_STAMP), xts(vixDt$CLOSE, vixDt$TIME_STAMP))
names(allXts) <- c('SP500', 'VIX')


In [3]:
allXts$SMA_200 <- SMA(allXts$SP500, 200)
allXts$D_RET <- dailyReturn(allXts$SP500)
allXts$D_RET_LAG_1 <- stats::lag(allXts$D_RET, -1) #signal is generated and traded at the close, so use next day's return

allXts$VIX_ADJ_RET <- allXts$D_RET/allXts$VIX
allXts$VIX_ADJ_RET_SMA_200 <- SMA(allXts$VIX_ADJ_RET, 200) 

allXts$ROC_200 <- rollapply(allXts$D_RET, 200, Return.cumulative) #simple compound return or ROC (rate of change) 

allXts <- na.omit(allXts)
print(head(allXts))
print(tail(allXts))

            SP500   VIX  SMA_200         D_RET   D_RET_LAG_1    VIX_ADJ_RET
1990-10-15 303.23 31.64 338.9269  0.0106656401 -0.0142136261  0.00033709357
1990-10-16 298.92 31.45 338.6230 -0.0142136261 -0.0005352703 -0.00045194358
1990-10-17 298.76 31.37 338.3230 -0.0005352703  0.0233631670 -0.00001706313
1990-10-18 305.74 29.60 338.0734  0.0233631670  0.0220449441  0.00078929618
1990-10-19 312.48 27.86 337.8748  0.0220449441  0.0072964635  0.00079127578
1990-10-22 314.76 27.27 337.6796  0.0072964635 -0.0076249362  0.00026756375
           VIX_ADJ_RET_SMA_200    ROC_200
1990-10-15     -0.000016508160 -0.1569685
1990-10-16     -0.000018767878 -0.1689510
1990-10-17     -0.000018142490 -0.1672427
1990-10-18     -0.000011955377 -0.1403830
1990-10-19     -0.000005573280 -0.1127768
1990-10-22     -0.000005349595 -0.1103197
             SP500   VIX  SMA_200        D_RET  D_RET_LAG_1    VIX_ADJ_RET
2019-06-20 2954.18 14.75 2775.676  0.009472185 -0.001259223  0.00064218205
2019-06-21 2950.46 15.40

In [4]:
#go long only if the index is above its 200-day average
allXts$SMA_STRAT <- ifelse(allXts$SP500 > allXts$SMA_200, allXts$D_RET_LAG_1, 0) 

#go long only if the simple average (VIX-Adjusted Momentum) > 0
allXts$VIX_STRAT <- ifelse(allXts$VIX_ADJ_RET_SMA_200 > 0, allXts$D_RET_LAG_1, 0) 

#go long only if the ROC > 0
allXts$ROC_200_STRAT <- ifelse(allXts$ROC_200 > 0, allXts$D_RET_LAG_1, 0)

In [5]:
#plot returns
toPlot <- merge(allXts$SMA_STRAT, allXts$VIX_STRAT, allXts$ROC_200_STRAT, allXts$SP500)
Common.PlotCumReturns(toPlot, "", "")

<simpleError in plot.window(c(655948800, 1561593600), c(NaN, NaN)): need finite 'ylim' values>


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)